In [ ]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

In [ ]:
dotenv_path = "../.env"

In [ ]:
load_dotenv(dotenv_path)

In [ ]:
# Spark Config
SPARK_APPNAME = "delta-minio-tests"
SPARK_MASTER = os.getenv("SPARK_MASTER")

# Minio Config
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")
MINIO_USER = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")

In [ ]:
spark = SparkSession.builder \
    .appName(SPARK_APPNAME) \
    .master(SPARK_MASTER) \
    .config("spark.jars.packages",
            "io.delta:delta-spark_2.13:4.0.0,"
            "org.apache.hadoop:hadoop-aws:3.4.1,") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_USER) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_PASSWORD) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", False) \
    .getOrCreate()

In [ ]:
df = spark.range(5)

In [ ]:
df.write.format("delta").mode("overwrite").save("s3a://my-bucket/delta-table")